In [21]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import json
import tensorflowjs as tfjs

## Import Text

In [22]:
with open("IndiaUS.txt", 'r', encoding='utf-8') as myfile:
    mytext = myfile.read()

## Tokenize Text

In [23]:
mytokenizer = Tokenizer()
mytokenizer.fit_on_texts([mytext])
total_words = len(mytokenizer.word_index) + 1

In [24]:
my_input_sequences = []
for line in mytext.split('\n'):
    token_list = mytokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        my_n_gram_sequence = token_list[:i+1]
        my_input_sequences.append(my_n_gram_sequence)

In [25]:
max_sequence_len = max([len(seq) for seq in my_input_sequences])
input_sequences = np.array(pad_sequences(my_input_sequences, maxlen=max_sequence_len, padding='pre'))

In [26]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [27]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

## Create Model Architecture

In [28]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 82, 100)           59900     
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 599)               90449     
                                                                 
Total params: 300949 (1.15 MB)
Trainable params: 300949 (1.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


## Train Model

In [29]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
43/43 [==============================] - 5s 77ms/step - loss: 6.1516 - accuracy: 0.0496
Epoch 2/100
43/43 [==============================] - 3s 77ms/step - loss: 5.8033 - accuracy: 0.0474
Epoch 3/100
43/43 [==============================] - 3s 77ms/step - loss: 5.7245 - accuracy: 0.0576
Epoch 4/100
43/43 [==============================] - 3s 77ms/step - loss: 5.6635 - accuracy: 0.0576
Epoch 5/100
43/43 [==============================] - 3s 78ms/step - loss: 5.5857 - accuracy: 0.0576
Epoch 6/100
43/43 [==============================] - 3s 80ms/step - loss: 5.4919 - accuracy: 0.0591
Epoch 7/100
43/43 [==============================] - 3s 78ms/step - loss: 5.3695 - accuracy: 0.0656
Epoch 8/100
43/43 [==============================] - 3s 77ms/step - loss: 5.2179 - accuracy: 0.0759
Epoch 9/100
43/43 [==============================] - 3s 77ms/step - loss: 5.0487 - accuracy: 0.0963
Epoch 10/100
43/43 [==============================] - 3s 77ms/step - loss: 4.8527 - accuracy: 0.1123

## Save Model for JS

In [31]:

output_dir = "./tfjs_models/model" 
tfjs.converters.save_keras_model(model, output_dir)

C:\Users\lusti\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Save word index

In [32]:
wordIndex = {}
for word in mytokenizer.word_index:
    wordIndex[word] = mytokenizer.word_index[word]

with open('wordIndex.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(wordIndex, ensure_ascii=False))

## Save Max Sequence Length

In [33]:
with open('max_sequence_len.txt', 'w') as f:
    f.write(str(max_sequence_len))